In [ ]:
%%capture
# Install required packages for unsloth and training
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install datasets huggingface_hub

In [ ]:
# Login to Hugging Face Hub
from huggingface_hub import login
login()

In [ ]:
from unsloth import FastLanguageModel
import torch

# Model configuration
max_seq_length = 2048 
dtype = None  # Auto detect
load_in_4bit = True  # Enable 4-bit quantization

# Load pre-trained model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-1.5B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Apply LoRA adapters for efficient fine-tuning
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

In [ ]:
from datasets import load_dataset

# Dataset configuration - replace with your dataset name
dataset_id = "nggiabao19/toeic-tutor-v1"

# Load dataset from Hugging Face
dataset = load_dataset(dataset_id, split = "train")

# Format prompts for training
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        # Structure: Instruction + Response with EOS token
        text = f"{instruction}\n\n### Response:\n{output}" + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts, }

# Apply formatting to dataset
dataset = dataset.map(formatting_prompts_func, batched = True)
print("Dataset loaded and formatted successfully!")

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# Initialize trainer with training arguments
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,  # Demo with 60 steps. For full training, use num_train_epochs = 1
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",  # Disable WandB to avoid hanging
    ),
)

# Start training
print("Starting training...")
trainer_stats = trainer.train()

In [ ]:
# Switch to inference mode for faster generation
FastLanguageModel.for_inference(model)

# Prepare test input
inputs = tokenizer(
[
    """Dưới đây là một câu hỏi TOEIC. Hãy đóng vai một gia sư vui tính.

### Câu hỏi:
The board of directors is _______ to announce the new CEO tomorrow.

### Các lựa chọn:
A. schedule
B. scheduled
C. schedules
D. scheduling

### Yêu cầu:
Trả về định dạng JSON bao gồm: định nghĩa, giải thích đúng sai."""
], return_tensors = "pt").to("cuda")

# Generate response
outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
print(tokenizer.batch_decode(outputs)[0])

In [ ]:
# Save model locally
model.save_pretrained("toeic_model_lora")
tokenizer.save_pretrained("toeic_model_lora")

# Push to Hugging Face Hub (optional)
HF_MODEL_REPO = "nggiabao19/toeic-lm-v1"

print("Pushing model to Hugging Face Hub...")
model.push_to_hub(HF_MODEL_REPO, token = True)  # Uses token from login
tokenizer.push_to_hub(HF_MODEL_REPO, token = True)
print(f"Model uploaded successfully to: https://huggingface.co/{HF_MODEL_REPO}")